In [1]:
!pip install -q -r requirements.txt

In [2]:
!apt-get update >/dev/null
!apt-get -y install python3-dev libsm-dev libxrender1 libxext6 zip git >/dev/null
!rm -rf /var/lib/apt/lists/*

In [ ]:
!git pull

In [ ]:
!git config --global user.email "ovidio.mallo@gmail.com"
!git config --global user.email "ovidio.mallo@gmail.com"

In [78]:
!git add analysis.ipynb && git commit -m "updated analysis notebook" && git push


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@c14ca925d780.(none)')


In [37]:
import glob
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import cv2
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

from train import create_model, create_criterion, evaluate
from dataset import TrainData, TrainDataset

In [32]:
input_dir = "/storage/kaggle/quickdraw"
image_size = 32
batch_size = 256
test_size = 0.1
train_on_unrecognized = True
num_categories_restriction = 68
num_workers = 8
pin_memory = True
use_dummy_image = False

In [33]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [40]:
model = create_model("cnn", input_size=image_size, num_classes=num_categories_restriction)
model.load_state_dict(torch.load("/storage/models/quickdraw/test/model.pth", map_location=device))

In [11]:
train_data = TrainData(
    input_dir,
    shard=0,
    test_size=test_size,
    train_on_unrecognized=train_on_unrecognized,
    num_categories_restriction=num_categories_restriction)

Reading data file '/storage/kaggle/quickdraw/train_simplified_shards/shard-0.npz'
Loaded 994325 samples
Time to load data of shard 0: 0:00:29.659513


In [16]:
val_set = TrainDataset(train_data.val_set_df, image_size, use_dummy_image)
val_set_data_loader = \
    DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)

In [20]:
criterion = create_criterion("cce", len(train_data.categories))

In [22]:
loss_avg, mapk_avg, accuracy_top1_avg, accuracy_top3_avg, accuracy_top5_avg, accuracy_top10_avg = \
    evaluate(model, val_set_data_loader, criterion, 3)

In [24]:
print(
    "loss: {:.2f}, map@3: {:.2f}, acc@1: {:.2f}, acc@3: {:.2f}, acc@5: {:.2f}, acc@10: {:.2f}"
    .format(loss_avg, mapk_avg, accuracy_top1_avg, accuracy_top3_avg, accuracy_top5_avg, accuracy_top10_avg))

loss: 4.22, map@3: 0.04, acc@1: 0.03, acc@3: 0.06, acc@5: 0.08, acc@10: 0.16


In [26]:
val_set_data_loader_iter = iter(val_set_data_loader)

In [34]:
batch = next(val_set_data_loader_iter)

images, categories = \
    batch[0].to(device, non_blocking=True), \
    batch[1].to(device, non_blocking=True)

In [41]:
prediction_logits = model(images)
predictions = F.softmax(prediction_logits, dim=1)

In [50]:
prediction_scores, prediction_categories = predictions.topk(3, dim=1, sorted=True)

In [76]:
print(prediction_scores[:10, 0])

tensor([0.9995, 0.6642, 0.5088, 0.9105, 1.0000, 0.9384, 0.9272, 0.9937, 0.9865,
        0.9923], device='cuda:0', grad_fn=<SelectBackward>)
